In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install fancyimpute

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 2.7 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29881 sha256=cfe65e6684437e14a7b82be6f9e3314b0db2ae0717672d4bd8639093d84cfa43
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11330 sha256=9c0144553a01876e406a8897c8b375dc54caf791b54ee890981e23af28ed962c
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif

In [5]:
# Read the data
PATH = '/content/drive/MyDrive/Dataset/'
df = pd.read_csv(PATH + 'csv_preprocessed.csv')
print(df.shape)

(1215675, 63)


In [6]:
# Define features and target
X = df.drop(columns=['Attack Type'])
y = df['Attack Type']

In [7]:
# Perform mutual information feature selection
mutual_info_arr = mutual_info_classif(X, y)
top_features = X.columns[np.argsort(mutual_info_arr)[::-1][:15]]
X_selected = X[top_features]

In [8]:
# Perform PCA
pca = PCA(n_components=15)
X_pca = pca.fit_transform(X_selected)

In [9]:
# Define the parameter grid - Part 2
param_grid = {
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

In [10]:
# Initialize Random Forest Classifier
rfc = RandomForestClassifier(random_state=42)

In [11]:
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, scoring='accuracy')

In [12]:
# Perform Grid Search
grid_search.fit(X_pca, y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'min_samples_leaf': [1, 2, 4]},
             scoring='accuracy')

In [13]:
# Get the best parameters
best_params_part2 = grid_search.best_params_

In [14]:
# Train the model with the best parameters from Part 2
best_rfc_part2 = RandomForestClassifier(**best_params_part2, random_state=42)
best_rfc_part2.fit(X_pca, y)

RandomForestClassifier(bootstrap=False, random_state=42)

In [15]:
# Predictions
y_pred_part2 = best_rfc_part2.predict(X_pca)

In [16]:
# Evaluate the model
accuracy_part2 = accuracy_score(y, y_pred_part2)
precision_part2 = precision_score(y, y_pred_part2, average='weighted')
recall_part2 = recall_score(y, y_pred_part2, average='weighted')
f1_part2 = f1_score(y, y_pred_part2, average='weighted')

In [17]:
# Print evaluation metrics
print(f'Accuracy: {accuracy_part2}')
print(f'Precision: {precision_part2}')
print(f'Recall: {recall_part2}')
print(f'F1 Score: {f1_part2}')

Accuracy: 0.9999629835276698
Precision: 0.9999630014546752
Recall: 0.9999629835276698
F1 Score: 0.9999629854914502
